In [ ]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [ ]:
base_url = "https://www.trustpilot.com/review/www.gotogate.com"
print(base_url)

https://www.trustpilot.com/review/www.gotogate.com


In [ ]:
header = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
    ),
    'Referer': 'https://www.trustpilot.com/review/www.gotogate.com'
}

In [ ]:
search_response=requests.get(base_url,headers=header)

In [ ]:
search_response.status_code

200

In [ ]:

def Searchreviews(page_no):
    url = f"https://www.trustpilot.com/review/www.gotogate.com?page={page_no}"
    print(f"Fetching: {url}")

    try:
        page = requests.get(url, headers=header)
        if page.status_code == 200:
            return page
        else:
            print(f"Status code {page.status_code} on page {page_no}")
            return "Error"
    except Exception as e:
        print(f"Request failed for page {page_no}: {e}")
        return "Error"

In [ ]:
reviews = []
ratings = []

for k in range(2, 600):
    try:
        response = Searchreviews(k)
        print(f"Response for page {k} fetched")

        if isinstance(response, str) and response == "Error":
            continue

        soup = BeautifulSoup(response.content, 'html.parser')

        # Review blocks
        review_blocks = soup.find_all("div", class_="styles_reviewContent__tuXiN")
        print(f"Found {len(review_blocks)} review blocks on page {k}")

        for block in review_blocks:
            # Get review text from <p> inside the review block
            p_tag = block.find("p", class_="typography_body-l__v5JLj typography_appearance-default__t8iAq")
            review_text = p_tag.text.strip() if p_tag else "No review text"

            # Go up and find the rating div above this review block
            rating_div = block.find_previous("div", class_="styles_reviewHeader__DzoAZ")
            rating_value = int(rating_div.get("data-service-review-rating", -1)) if rating_div else -1

            print(f"Page {k}: {review_text} | Rating: {rating_value}")
            reviews.append(review_text)
            ratings.append(rating_value)

    except Exception as e:
        print(f"Error on page {k}: {e}")
        continue


Streaming output truncated to the last 5000 lines.
Page 168: Delay with answering emailI had to wait soo long for answer from them, especially when time is money and there was a change in the details.They should filter their support by urgency. | Rating: 1
Page 168: Easy and fast | Rating: 5
Page 168: We bought plane tickets from go to gate to croatia. Then one of the connection planes got cancelled and we contaced the company many many times and after several times we finally got a replacing flight route. We payed and everything But never got the confirmation for the tickets. After several months they finally contact us back and to say the tickets we bought was suddenly sold out. After several months!! Now it’s only less than two months until we r supposed to travel. And therefore twice or trippel as expensive for plane tickets. I do not recommend go to gate. Bad service bad company. | Rating: 1
Page 168: Company sells you flexible tickets and says their support is helpful only to be 

In [ ]:
print("\nTotal reviews collected:", len(reviews))
print("Total ratings collected:", len(ratings))


Total reviews collected: 7040
Total ratings collected: 7040


In [ ]:
import pandas as pd
review_data = pd.DataFrame({
    'Review': reviews,
    'Rating': ratings
})

review_data.to_csv('reviews_and_ratings.csv', index=False)

from google.colab import files
files.download('reviews_and_ratings.csv')

print(review_data.head())


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

                                              Review  Rating
0                                     No review text       5
1  Flight was cancelled and no refund provided. H...       1
2                                     No review text       5
3  I had one of the worst travel experiences ever...       1
4  No problem at all. Cheap, easy, fast, just ski...       5


In [ ]:
import pandas as pd
from google.colab import files

# Step 1: Read the original reviews and ratings
df = pd.read_csv('reviews_and_ratings.csv')

# Step 2: Add Sentiment column
df['Sentiment'] = df['Rating'].apply(lambda x: 'Negative' if x < 3 else ('Positive' if x > 3 else 'Neutral'))

# Step 3: Save the updated DataFrame to a new CSV file
df.to_csv('reviews_with_sentiment.csv', index=False)

# Step 4: Download the CSV file to your local system
files.download('reviews_with_sentiment.csv')

# Optional: Confirm it's done
print("✅ Sentiment column added and CSV downloaded successfully!")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Sentiment column added and CSV downloaded successfully!


In [ ]:
# Count sentiment types
sentiment_counts = df['Sentiment'].value_counts()

# Print results
print("📊 Sentiment Distribution:")
print(sentiment_counts)


📊 Sentiment Distribution:
Sentiment
Negative    3716
Positive    3132
Neutral      192
Name: count, dtype: int64


In [ ]:
!pip install nltk spacy
!python -m nltk.downloader stopwords punkt
!python -m spacy download en_core_web_sm

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import nltk
import spacy
nltk.download('punkt')         # For tokenization
nltk.download('stopwords')# For stopwords removal
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load your data
df = pd.read_csv("reviews_with_sentiment.csv")  # Use your actual filename

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Load spaCy model for lemmatization
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Remove punctuation, numbers, special characters
    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenization
    tokens = word_tokenize(text)

    # 4. Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # 5. Lemmatization using spaCy
    doc = nlp(" ".join(filtered_tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]

    return " ".join(lemmatized_tokens)

# Apply to your reviews column
df['cleaned_review'] = df['Review'].astype(str).apply(preprocess_text)

# Save cleaned data
df.to_csv("reviews_cleaned.csv", index=False)

print("Text preprocessing complete. Cleaned file saved as 'reviews_cleaned.csv'.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Text preprocessing complete. Cleaned file saved as 'reviews_cleaned.csv'.


In [ ]:
from google.colab import files

# Download the cleaned file
files.download("reviews_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
df = pd.read_csv("reviews_cleaned.csv")  # Use your cleaned file
texts = df['cleaned_review'].fillna('')  # Just in case there are NaNs

In [ ]:
# Initialize CountVectorizer (BoW)
bow_vectorizer = CountVectorizer()

# Transform the text
bow_features = bow_vectorizer.fit_transform(texts)

# Convert to DataFrame for better visualization
bow_df = pd.DataFrame(bow_features.toarray(), columns=bow_vectorizer.get_feature_names_out())

print("Bag of Words shape:", bow_df.shape)
bow_df.head()

Bag of Words shape: (7040, 13000)


,aa,aaaaaaaaaasasssss,aal,aan,aaron,ab,ababa,abandon,abbreviate,abd,...,zo,zojuist,zone,zoo,zoom,zorgplicht,zu,zum,zurck,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transform the text
tfidf_features = tfidf_vectorizer.fit_transform(texts)

# Convert to DataFrame for better visualization
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print("TF-IDF shape:", tfidf_df.shape)
tfidf_df.head()

TF-IDF shape: (7040, 13000)


,aa,aaaaaaaaaasasssss,aal,aan,aaron,ab,ababa,abandon,abbreviate,abd,...,zo,zojuist,zone,zoo,zoom,zorgplicht,zu,zum,zurck,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
bow_df.to_csv("bag_of_words.csv", index=False)
print("Bag of Words saved as bag_of_words.csv")

Bag of Words saved as bag_of_words.csv


In [ ]:
tfidf_df.to_csv("tfidf_features.csv", index=False)
print("TF-IDF features saved as tfidf_features.csv")

TF-IDF features saved as tfidf_features.csv


In [ ]:
# Optionally, you can download the files from Google Colab
from google.colab import files
files.download("bag_of_words.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Optionally, you can download the files from Google Colab
from google.colab import files

files.download("tfidf_features.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("reviews_cleaned.csv")

df = df.dropna(subset=['Review', 'Sentiment'])

X = df['Review']
y = df['Sentiment']

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred))

✅ Accuracy: 0.9318181818181818

📋 Classification Report:
               precision    recall  f1-score   support

    Negative       0.93      0.96      0.95       755
     Neutral       0.00      0.00      0.00        33
    Positive       0.93      0.94      0.94       620

    accuracy                           0.93      1408
   macro avg       0.62      0.64      0.63      1408
weighted avg       0.91      0.93      0.92      1408



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict_sentiment(review_text):
    review_tfidf = tfidf.transform([review_text])
    prediction = model.predict(review_tfidf)[0]
    return prediction

In [ ]:
user_input = input("Enter a review: ")
predicted_sentiment = predict_sentiment(user_input)
print(f"Predicted Sentiment: {predicted_sentiment}")

Enter a review: I had one of the worst travel experiences ever thanks to GoToGate. We nearly missed our flight due to an absurdly long and poorly planned connection. For over an hour, we had to run with our luggage”through a bus transfer, a train, immigration, passport control, and two security checks.
Predicted Sentiment: Negative


In [ ]:
# Define keywords for each category
complaint_keywords = {
    'Refund Issues': ['refund', 'money back', 'not received', 'reimbursement'],
    'Customer Service': ['support', 'help', 'rude', 'not responding', 'contact'],
    'Booking Issues': ['booking', 'ticket', 'confirmation', 'reservation'],
    'Scam or Fraud': ['scam', 'fraud', 'cheat', 'dishonest', 'fake'],
    'Delays': ['delay', 'late', 'waiting', 'postponed'],
    'Technical Issues': ['website', 'crash', 'app', 'error', 'technical'],
}


In [ ]:
def assign_category(text):
    # Ensure text is a string and not NaN
    if isinstance(text, str):
        text = text.lower()
        for category, keywords in complaint_keywords.items():
            for keyword in keywords:
                if keyword in text:
                    return category
    return 'Other'


In [ ]:
# Assuming df['cleaned_review'] exists. If not, use df['Review']
df['Complaint_Category'] = df['cleaned_review'].apply(assign_category)


In [ ]:
df.to_csv('reviews_with_categories.csv', index=False)
from google.colab import files
files.download('reviews_with_categories.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>